In [ ]:
EC2 Docs
https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/concepts.html

EC2 On Demand Prices
https://aws.amazon.com/ec2/pricing/on-demand/

EC2 Spot Instances Prices
EC2 dashboard -> Instances -> Spot Request
https://console.aws.amazon.com/ec2sp/v1/spot/home?region=us-east-1#

https://aws.amazon.com/ec2/spot/pricing/


#################

#!/bin/bash

cd ~

# ami-6637e51b
#Deep Learning AMI (Amazon Linux) Version 6.0 - ami-6637e51b

echo Starting provisioning $(date +%Y-%m-%d_%H%M%S)

#update VM

sudo yum -y update

#configure Jupyter server
#https://docs.aws.amazon.com/mxnet/latest/dg/setup-jupyter-configure-server.html

if [[ $USER =~ 'ec2-user' ]] && [[ ! -d ssl ]]; then
    echo Making directory 'ssl'
    mkdir ssl
    cd ssl
    sudo openssl req -x509 -nodes -days 365 -newkey rsa:1024 -keyout "cert.key" -out "cert.pem" -batch
fi

cd ~

if [[ $USER =~ 'ec2-user' ]] && [[ ! -d .jupyter ]]; then

    echo Making directory '.jupyter'
    mkdir .jupyter
    cd .jupyter
    
    jupyter notebook --generate-config

    cat << EOT >> jupyter_notebook_config.py
    #
    #https://docs.aws.amazon.com/mxnet/latest/dg/setup-jupyter-configure-server.html
    #
    c = get_config()  # Get the config object.
    c.NotebookApp.certfile = u'/home/ec2-user/ssl/cert.pem' # path to the certificate we generated
    c.NotebookApp.keyfile = u'/home/ec2-user/ssl/cert.key' # path to the certificate key we generated
    c.IPKernelApp.pylab = 'inline'  # in-line figure when using Matplotlib
    c.NotebookApp.ip = '*'  # Serve notebooks locally.
    c.NotebookApp.open_browser = False  # Do not open a browser window by default when using notebooks.
    c.NotebookApp.password = 'sha1:a246bbba6b51:5b18c74a2fe2ba37b48c403ab25bd66defe76463'
    EOT
    
fi

cd ~

#conda install -y anaconda
#for TensorFlow(+Keras2) with Python3 (CUDA 9/MKL) ____________ source activate tensorflow_p36
source activate tensorflow_p36

git clone https://github.com/naturomics/CapsNet-Tensorflow.git

cd CapsNet-Tensorflow

pip3 install --upgrade tqdm

#jupyter notebook&

######
#
#On client:
# ssh -i ~/mykeypair.pem -L 8157:127.0.0.1:8888 ec2-user@ec2-###-##-##-###.compute-1.amazonaws.com
# jupyter notebook &
#In browser goto:
# https://127.0.0.1:8157
#
######

out=${user}_$(date +%Y-%m-%d_%H%M%S).out
err=${user}_$(date +%Y-%m-%d_%H%M%S).err

python download_data.py
python download_data.py --dataset fashion-mnist --save_to data/fashion-mnist
nohup python main.py --epoch=5 1>$out 2>$err 0>/dev/null &

echo 'main.py' launched with PID $! at $(date +%Y-%m-%d_%H%M%S)
exit



<b><h3>Margin Loss</h3></b>
![margin_loss](images/margin_loss.png)
<b><h3>Reconstruction Loss</h3></b>
![reconstruction_loss](images/reconstruction_loss.png)
<b><h3>Total Loss</h3></b>
![total_loss](images/total_loss.png)

<b><h3>Routing Trials</h3></b>
![routing_trials](images/routing_trials.png)

<b><h3>Reconstructed Images</h3></b>
![recon_5](images/recon_5.png)![recon_8](images/recon_8.png)![recon_9](images/recon_9.png)